In [72]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘edgar_allan_poe.txt’ already there; not retrieving.

File ‘robert_frost.txt’ already there; not retrieving.



In [73]:
#Given 2 sets of files containing poems by edgar allen poe & robert frost 
#If now given an unkown poem find the probable author

frost = []
poe = []
alphabets = [' ','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

with open('robert_frost.txt') as file:
  for line in file.readlines(): 
    line = line.replace('\n','')
    line = line.lower()
    for ch in line:
      if ch not in alphabets:
        line = line.replace(ch,'')
    if len(line)!=0:
      frost.append(line.split())

with open('edgar_allan_poe.txt') as file:
  for line in file.readlines(): 
    line = line.replace('\n','')
    line = line.lower()
    for ch in line:
      if ch not in alphabets:
        line = line.replace(ch,'')
    if len(line)!=0:
      poe.append(line.split())

In [74]:
inputs = frost+poe
labels = ['frost']*len(frost) + ['poe']*len(poe)

In [75]:
train_text, test_text, Ytrain, Ytest = train_test_split(inputs, labels)
print(len(train_text),len(Ytrain),len(test_text))
print(Ytrain[:5])
train_text[:5]

1615 1615 539
['frost', 'poe', 'frost', 'poe', 'poe']


[['ill', 'show', 'you', 'how', 'she', 'had', 'her', 'sills', 'extended'],
 ['to', 'the', 'loved', 'object', 'so', 'the', 'tear', 'to', 'the', 'lid'],
 ['you', 'see', 'they', 'know', 'i', 'havent', 'had', 'your', 'name'],
 ['this', 'ghoulhaunted', 'woodland', 'of', 'weir'],
 ['in', 'beauty', 'by', 'our', 'god', 'to', 'those', 'alone']]

In [76]:
train_text[-1]

['the', 'playthings', 'in', 'the', 'playhouse', 'of', 'the', 'children']

In [77]:
idx=1
word2idx = {'<unk>': 0}
idx2word = {0:'<unk>'}

for line in train_text:
    for token in line:
      if token not in word2idx:
        word2idx[token] = idx
        idx2word[idx] = token
        idx += 1

In [78]:
word2idx

{'<unk>': 0,
 'ill': 1,
 'show': 2,
 'you': 3,
 'how': 4,
 'she': 5,
 'had': 6,
 'her': 7,
 'sills': 8,
 'extended': 9,
 'to': 10,
 'the': 11,
 'loved': 12,
 'object': 13,
 'so': 14,
 'tear': 15,
 'lid': 16,
 'see': 17,
 'they': 18,
 'know': 19,
 'i': 20,
 'havent': 21,
 'your': 22,
 'name': 23,
 'this': 24,
 'ghoulhaunted': 25,
 'woodland': 26,
 'of': 27,
 'weir': 28,
 'in': 29,
 'beauty': 30,
 'by': 31,
 'our': 32,
 'god': 33,
 'those': 34,
 'alone': 35,
 'whose': 36,
 'shoes': 37,
 'primped': 38,
 'dance': 39,
 'but': 40,
 'have': 41,
 'said': 42,
 'it': 43,
 'and': 44,
 'were': 45,
 'off': 46,
 'find': 47,
 'theres': 48,
 'not': 49,
 'a': 50,
 'soul': 51,
 'leaves': 52,
 'crisped': 53,
 'sere': 54,
 'perhaps': 55,
 'may': 56,
 'be': 57,
 'that': 58,
 'my': 59,
 'mind': 60,
 'is': 61,
 'wrought': 62,
 'from': 63,
 'their': 64,
 'throats': 65,
 'old': 66,
 'man': 67,
 'seriously': 68,
 'took': 69,
 'his': 70,
 'bearings': 71,
 'smoke': 72,
 'rolled': 73,
 'inside': 74,
 'sockets': 75

In [79]:
len(word2idx)

2525

In [80]:
train_text_int = []
test_text_int = []

for tokens in train_text:
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  train_text_int.append(line_as_int)

for tokens in test_text:
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [81]:
from sklearn.model_selection import train_test_split
import numpy as np

M = len(word2idx) 
e = 1

pi_frost = np.ones(M)
A_frost = np.ones((M,M))

pi_poe = np.ones(M)
A_poe = np.ones((M,M))


def compute_counts(corpus,A,pi):
  for line in corpus:
    #print(line)
    pi[line[0]]+=1
    for i in range(1,len(line)):
      #print(A[word2idx[line[i-1]]][word2idx[line[i]]])
      #print(line[i-1], line[i], word2idx[line[i-1]],word2idx[line[i]])
      A[line[i-1],line[i]]+=1

compute_counts([line for line,author in zip(train_text_int,Ytrain) if author=='frost'], A_frost, pi_frost)
compute_counts([line for line,author in zip(train_text_int,Ytrain) if author=='poe'], A_poe, pi_poe)

In [82]:
A_frost /= A_frost.sum(axis=1, keepdims=True) 
A_poe /= A_poe.sum(axis=1, keepdims=True)

pi_frost /= pi_frost.sum()
pi_poe /= pi_poe.sum()

In [83]:
logA_frost = np.log(A_frost)
logpi_frost = np.log(pi_frost)

logA_poe = np.log(A_poe)
logpi_poe = np.log(pi_poe)

In [84]:
# compute priors
count_frost = sum(y == 'frost' for y in Ytrain)
count_poe = sum(y == 'poe' for y in Ytrain)
total = len(Ytrain)
p_frost = count_frost / total
p_poe = count_poe / total
logp_frost = np.log(p_frost)
logp_poe = np.log(p_poe)
p_frost, p_poe
#If classes are imbalanced we should not use the maximum likelihood method

(0.6730650154798762, 0.3269349845201238)

In [85]:
#build a classifier
def _compute_log_likelihood(line,logA,logpi):
  # p(s1,s2,s3,...,sT) = p(s1)*p(s2|s1)*p(s3|s1,s2)*...*p(sT|s1,s2,...,sT-1)
  #                    = p(s1)*p(s2/s1)*p(s3/s2)*....p(sT|sT-1)
  #                    = pi(s1)*As1s2*As2s3*....*AsT-1sT 
  # log( p(s1,s2,s3,...,sT) ) = log( pi (s1)) + Σ t = 2 to T ( log (Ast-1st) )
  logp = logpi[line[0]]
  for i in range(1, len(line)):
    logp += logA[line[i-1],line[i]]
  return logp

def predict(text_int):
  predictions = []
  for line in text_int:
    #we have to predict who is the poet of this line
    p_frost = _compute_log_likelihood(line,logA_frost,logpi_frost)+logp_frost
    p_poe = _compute_log_likelihood(line,logA_poe,logpi_poe)+logp_poe
    if p_frost > p_poe:
      predictions.append('frost')
    else:
      predictions.append('poe')
  return predictions 

In [86]:
Ptrain = predict(train_text_int)
correct = 0
for pred,real in zip(Ptrain,Ytrain):
  if pred == real:
    correct += 1
print(f"Train accuracy : {correct/len(Ytrain)*100}")

Train accuracy : 99.38080495356037


In [87]:
Ptest = predict(test_text_int)
correct = 0
for pred,real in zip(Ptest,Ytest):
  if pred == real:
    correct += 1
print(f"Test accuracy : {correct/len(Ytest)*100}")

Test accuracy : 79.77736549165121


In [88]:
from sklearn.metrics import confusion_matrix, f1_score
# read about F-score: https://en.wikipedia.org/wiki/F-score

In [89]:
for i in range(len(Ptrain)):
  if Ptrain[i]=='frost':
    Ptrain[i]=0
  else:
    Ptrain[i]=1

for i in range(len(Ytrain)):
  if Ytrain[i]=='frost':
    Ytrain[i]=0
  else:
    Ytrain[i]=1

for i in range(len(Ptest)):
  if Ptest[i]=='frost':
    Ptest[i]=0
  else:
    Ptest[i]=1

for i in range(len(Ytest)):
  if Ytest[i]=='frost':
    Ytest[i]=0
  else:
    Ytest[i]=1

In [90]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[1087,    0],
       [  10,  518]])

In [91]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[339,  10],
       [ 99,  91]])

In [92]:
f1_score(Ytrain, Ptrain)

0.9904397705544934

In [93]:
f1_score(Ytest, Ptest)

0.6254295532646048